This is a code for VARIMA model

In [ ]:
import pandas as pd
from statsmodels.tsa.statespace.varmax import VARMAX
from sklearn.model_selection import TimeSeriesSplit
from itertools import product
import warnings

# Load your dataset
df = pd.read_csv('BR_param_norm.csv', index_col='date', parse_dates=['date'])

# Define the range of parameters to test
p = d = q = range(0, 3)  # Example ranges, adjust based on your dataset
P = D = Q = range(0, 2)  # Example ranges for seasonal components
m = [12]  # Seasonal period, adjust based on your dataset
parameters = product(p, d, q, P, D, Q, m)
parameters_list = list(parameters)

# Define the time series cross-validator
tscv = TimeSeriesSplit(n_splits=3)

# Function to run the grid search
def run_grid_search(df, parameters_list):
    best_aic = float('inf')
    best_params = None
    results = []
    
    for param in parameters_list:
        try:
            model = VARMAX(df, order=(param[0], param[1], param[2]), seasonal_order=(param[3], param[4], param[5], param[6]), enforce_stationarity=False, enforce_invertibility=False)
            for train_index, test_index in tscv.split(df):
                train, test = df.iloc[train_index], df.iloc[test_index]
                model_fit = model.fit(disp=False)
                aic = model_fit.aic
                if aic < best_aic:
                    best_aic = aic
                    best_params = param
                results.append((param, model_fit.aic))
                print(f'Tested: {param}, AIC: {aic}')
        except Exception as e:
            continue
    
    return best_params, results

# Run the grid search
warnings.filterwarnings("ignore")  # Ignore convergence warnings for demonstration
best_params, results = run_grid_search(df, parameters_list)

print(f'Best Parameters: {best_params}')

In [ ]:
import pandas as pd
from statsmodels.tsa.api import VAR
from statsmodels.tsa.statespace.varmax import VARMAX
from sklearn.model_selection import train_test_split

# Load the dataset
file_path = 'BR_param_norm.csv'
df = pd.read_csv(file_path, index_col='date', parse_dates=['date'])

# Ensure your data is stationary before proceeding

# Split the data into train, validation, and test sets
train_val, test = train_test_split(df, test_size=0.2, shuffle=False)
train, validation = train_test_split(train_val, test_size=0.25, shuffle=False)  # 0.25 x 0.8 = 0.2

# Model Selection: Fit a VAR model to determine optimal lag order for VARIMA
model_var = VAR(train)
lag_order = model_var.select_order(maxlags=12)
print(lag_order.summary())

# Assuming you've determined the best (p, d, q) and (P, D, Q, m) for your VARIMA model
# Fit a VARIMA model
model_varima = VARMAX(train, order=(p,d,q), seasonal_order=(P,D,Q,m), trend='c')
results_varima = model_varima.fit(disp=False)

# Display the model summary
print(results_varima.summary())

# Forecast on the validation set
forecast = results_varima.forecast(steps=len(validation))

# Evaluate the forecast (e.g., using RMSE)
from sklearn.metrics import mean_squared_error
from math import sqrt

rmse = sqrt(mean_squared_error(validation, forecast))
print(f'Validation RMSE: {rmse}')

# Forecast on the test set using the model fitted on the combined training and validation set
combined_train_val = pd.concat([train, validation])
model_varima_refit = VARMAX(combined_train_val, order=(p,d,q), seasonal_order=(P,D,Q,m), trend='c').fit(disp=False)
test_forecast = model_varima_refit.forecast(steps=len(test))

# Evaluate the test forecast (e.g., using RMSE)
test_rmse = sqrt(mean_squared_error(test, test_forecast))
print(f'Test RMSE: {test_rmse}')